In [ ]:
%run '/content/drive/MyDrive/Colab Notebooks/Hamoye Intership/classification/log_regression.ipynb'

In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(log_reg, normalized_train, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.45545128, 0.49872123, 0.44994499, 0.40241449, 0.51391162])

k-Fold Crosss Validation

In [18]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
kf = KFold(n_splits=5)
kf.split(normalized_train)
f1_scores = []

def _run_():
  for train_index, test_index in kf.split(normalized_train):
    x_train, x_test = [normalized_train.iloc[train_index], 
                        normalized_train.iloc[test_index]]
    y_train, y_test = [y_balanced.iloc[train_index], 
                        y_balanced.iloc[test_index]]
    model = LogisticRegression().fit(x_train, y_train)
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)
  return f1_scores

_run_()

[58.156028368794324,
 60.99290780141844,
 47.36842105263157,
 53.03030303030303,
 0.0]

Stratified K-Fold Cross Validation